In [0]:
%pip install requests==2.32.3 beautifulsoup4==4.12.3

Python interpreter will be restarted.
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Not uninstalling requests at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-7e5144e8-dc77-42fe-8309-b980b6022557
    Can't uninstall 'requests'. No files were found to uninstall.
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.1
    Not uninstalling beautifulsoup4 at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-7e5144e8-dc77-42fe-8309-b980b6022557
    Can't uninstall 'beautifulsoup4'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:
%pip install selenium==4.18.1 webdriver-manager==4.0.2 typing_extensions==4.12.2

Python interpreter will be restarted.
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-7e5144e8-dc77-42fe-8309-b980b6022557
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
  Attempting uninstall: attrs
    Found existing installation: attrs 21.4.0
    Not uninstalling attrs at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-7e5144e8-dc77-42fe-8309-b980b6022557
    Can't uninstall 'attrs'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:
%sh
sudo apt-get update
sudo apt-get install -y wget unzip
wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
sudo apt-get install -y ./google-chrome-stable_current_amd64.deb
sudo apt-get install -y -f  # Fix any dependency issues

Hit:1 https://repos.azul.com/zulu/deb stable InRelease
Hit:2 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
unzip is already the newest version (6.0-25ubuntu1.2).
wget is already the newest version (1.20.3-1ubuntu2.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
--2025-06-11 10:15:38--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.251.33.78, 2607:f8b0:400a:800::200e
Connecting to dl.google.com (dl.google.com)|142.251.33.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117322064 (112M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’



In [0]:
%sh
google-chrome --version

Google Chrome 137.0.7151.103 


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType
from pyspark.sql.functions import col
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

# Initialize Spark session
spark = SparkSession.builder.appName("WebScrapeTablesSelenium").getOrCreate()

# Function to clean numeric values
def clean_numeric(value):
    try:
        # Remove spaces and convert to float
        cleaned = value.replace(" ", "").replace(",", ".")
        return float(cleaned) if cleaned else 0.0
    except ValueError:
        return 0.0

# Function to scrape and parse table data using Selenium
def scrape_and_parse_table(url, table_index, table_name):
    # Set up Selenium with headless Chrome
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    # Explicitly set Chrome binary path
    chrome_options.binary_location = "/usr/bin/google-chrome"
    
    # Initialize Chrome driver
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
    
    try:
        # Load webpage
        driver.get(url)
        # Wait for JavaScript to render
        driver.implicitly_wait(10)
        
        # Get page source and parse with BeautifulSoup
        soup = BeautifulSoup(driver.page_source, "html.parser")
        tables = soup.find_all("table")
        
        # Debug: Print number of tables found
        print(f"Found {len(tables)} tables for {table_name}")
        
        if table_index >= len(tables):
            print(f"Table index {table_index} not found. Found {len(tables)} tables.")
            return None
        
        table = tables[table_index]
        rows = table.find_all("tr")
        headers = [th.get_text().strip().replace(" ", "_") for th in rows[0].find_all("th")][2:]  # Skip first two columns
        
        data = []
        if table_name == "prices_volumes":
            for row in rows[1:]:
                cells = [td.get_text().strip() for td in row.find_all("td")]
                metric = cells[0]
                values = cells[2:]  # Skip first two columns
                for date, value in zip(headers, values):
                    data.append((metric, date, clean_numeric(value)))
            
            schema = StructType([
                StructField("metric", StringType(), False),
                StructField("date", StringType(), False),
                StructField("value", FloatType(), False)
            ])
            df = spark.createDataFrame(data, schema)
            return df
        
        elif table_name == "block_products":
            for row in rows[1:]:
                cells = [td.get_text().strip() for td in row.find_all("td")]
                product = cells[0]
                values = cells[2:]  # Skip first two columns
                for date, value in zip(headers, values):
                    data.append((product, date, clean_numeric(value)))
            
            schema = StructType([
                StructField("product", StringType(), False),
                StructField("date", StringType(), False),
                StructField("price", FloatType(), False)
            ])
            df = spark.createDataFrame(data, schema)
            return df
        
        elif table_name == "hourly_products":
            current_hour = ""
            for row in rows[1:]:
                cells = [td.get_text().strip() for td in row.find_all("td")]
                if cells[0]:  # New hour
                    current_hour = cells[0]
                metric = cells[1]
                values = cells[2:]  # Skip first two columns
                for date, value in zip(headers, values):
                    data.append((current_hour, metric, date, clean_numeric(value)))
            
            schema = StructType([
                StructField("hour", StringType(), False),
                StructField("metric", StringType(), False),
                StructField("date", StringType(), False),
                StructField("value", FloatType(), False)
            ])
            df = spark.createDataFrame(data, schema)
            return df
    
    finally:
        driver.quit()

# URL to scrape
url = "https://ibex.bg/markets/dam/day-ahead-prices-and-volumes-v2-0-2/"

# Process and save each table
prices_volumes_df = scrape_and_parse_table(url, 0, "prices_volumes")
if prices_volumes_df:
    prices_volumes_df.write.mode("overwrite").format("delta").saveAsTable("prices_volumes_table")
    display(prices_volumes_df)

block_products_df = scrape_and_parse_table(url, 1, "block_products")
if block_products_df:
    block_products_df.write.mode("overwrite").format("delta").saveAsTable("block_products_table")
    display(block_products_df)

hourly_products_df = scrape_and_parse_table(url, 2, "hourly_products")
if hourly_products_df:
    hourly_products_df.write.mode("overwrite").format("delta").saveAsTable("hourly_products_table")
    display(hourly_products_df)

Found 4 tables for prices_volumes


metric,date,value
Prices (EUR/MWh),"Fri,_06/06",90.57
Prices (EUR/MWh),"Sat,_06/07",81.38
Prices (EUR/MWh),"Sun,_06/08",63.54
Prices (EUR/MWh),"Mon,_06/09",77.97
Prices (EUR/MWh),"Tue,_06/10",83.58
Prices (EUR/MWh),"Wed,_06/11",83.99
Volume (MWh),"Fri,_06/06",64425.5
Volume (MWh),"Sat,_06/07",65968.3
Volume (MWh),"Sun,_06/08",72070.8
Volume (MWh),"Mon,_06/09",69105.4


Found 4 tables for block_products


product,date,price
Base (01-24),"Fri,_06/06",90.57
Base (01-24),"Sat,_06/07",81.38
Base (01-24),"Sun,_06/08",63.54
Base (01-24),"Mon,_06/09",77.97
Base (01-24),"Tue,_06/10",83.58
Base (01-24),"Wed,_06/11",83.99
Peak (9-20),"Fri,_06/06",63.68
Peak (9-20),"Sat,_06/07",48.62
Peak (9-20),"Sun,_06/08",34.44
Peak (9-20),"Mon,_06/09",43.5


Found 4 tables for hourly_products


hour,metric,date,value
0 - 1,EUR/MWh,"Thu,_06/05",120.77
0 - 1,EUR/MWh,"Fri,_06/06",103.57
0 - 1,EUR/MWh,"Sat,_06/07",103.83
0 - 1,EUR/MWh,"Sun,_06/08",96.01
0 - 1,EUR/MWh,"Mon,_06/09",99.12
0 - 1,EUR/MWh,"Tue,_06/10",89.91
0 - 1,EUR/MWh,"Wed,_06/11",81.11
MWh,2330.2,"Thu,_06/05",2571.2
MWh,2330.2,"Fri,_06/06",2284.9
MWh,2330.2,"Sat,_06/07",2217.9


In [0]:
%sql
SELECT * FROM prices_volumes_table LIMIT 5;
SELECT * FROM block_products_table LIMIT 5;
SELECT * FROM hourly_products_table LIMIT 5;

hour,metric,date,value
0 - 1,EUR/MWh,"Thu,_06/05",120.77
0 - 1,EUR/MWh,"Fri,_06/06",103.57
0 - 1,EUR/MWh,"Sat,_06/07",103.83
0 - 1,EUR/MWh,"Sun,_06/08",96.01
0 - 1,EUR/MWh,"Mon,_06/09",99.12
